In [ ]:
from bs4 import BeautifulSoup
import requests
from htmldate import find_date
import pandas as pd
def scrape(url):
    try:
        response = requests.get(url)
    except Exception:
        return "DELETE"
    html_content = response.text
    soup = BeautifulSoup(html_content, 'lxml')  # You can also use 'html.parser'
    paragraphs = soup.find_all("p")
    textOut = ""
    timeStamp = ""
    try:
        timeStamp = find_date(url)
        year = timeStamp.split("-")[0]
        month = timeStamp.split("-")[1]
        day = timeStamp.split("-")[2]
    except Exception:
        year = "check"
        month = "check"
        day = "check"
    captions = soup.find_all("span")
    otherRemove = soup.find_all("div",{"class":"GenericCalloutWrapper-tojWn bPlNqk callout--has-top-border"})
    oRemoveList = []
    for o in otherRemove:
        oRemoveList.append(o.get_text().replace("\n",""))
    capList = []
    lastspan = ""
    if len(captions) != 0:
        for c in captions:
            caption = c.get_text().replace("\n","")
            if caption =="":
                continue
            if caption[-1] in "!.?":
                capList.append(caption)
                continue
            if "Courtesy of" in caption or "Photograph" in caption:
                if lastspan not in capList:
                    capList.append(lastspan)
            lastspan = caption
    about = False
    
    removeList = ["© 2024 Condé Nast. All rights reserved. WIRED may earn a portion of sales from products that are purchased through our site as part of our Affiliate Partnerships with retailers. The material on this site may not be reproduced, distributed, transmitted, cached or otherwise used, except with the prior written permission of Condé Nast. Ad Choices",
                    "More From WIRED","Reviews and Guides","To revisit this article, visit My Profile, then View saved stories.","Related Wired coverage here.","If you buy something using links in our stories, we may earn a commission. Learn more.","WIRED Opinion publishes articles by outside contributors representing a wide range of viewpoints. Read more opinions here, and see our submission guidelines here. Submit an op-ed at opinion@wired.com.",
                    "SUPPORT REQUEST :"]  
    for p in paragraphs:
        about = False
    
        text = p.get_text().replace("\n","").strip()
        #print(text)
        if "Scoop:" in text  or "How did coronavirus start and what happens next?" in text or "More From WIRED" == text:
            break
        for j in oRemoveList:
            if text in j:
                about = True
                break
        if len(text) >1 and text not in removeList and text[-1] in '!.?:”))"' and "Read more: " not in text and text not in capList and not about:
            if paragraphs.index(p)< len(paragraphs)-1:
                nextLine = paragraphs[paragraphs.index(p)+1].get_text().replace("\n","").strip()
            if (paragraphs.index(p)< len(paragraphs)-1 and "Updated, " in text and (nextLine =="More From WIRED" or "Updated, " in  nextLine)) or (paragraphs.index(p)< len(paragraphs)-1 and "Correction: " in text and ("Scoop:" in nextLine or nextLine =="More from WIRED")):
                break
            textOut+= " " + text
    length = 0
    if textOut == "":
        return "DELETE"
    else:
        length = len(textOut.split())
        return [url,"Wired",year,month,day,length,textOut]
df = []
